# Part 1

In [ ]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "pierre!!"

In [ ]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

In [ ]:
edges = {
    '.':{'N':['S'], 'S':['N'], 'E':['W'], 'W':['E']},
    '-':{'N':['E','W'], 'S':['E','W'], 'E':['W'], 'W':['E']},
    '|':{'N':['S'], 'S':['N'], 'E':['N','S'], 'W':['N','S']},
    '/':{'N':['W'], 'S':['E'], 'E':['S'], 'W':['N']},
    '\\':{'N':['E'], 'S':['W'], 'E':['N'], 'W':['S']}
}

In [ ]:
gds.run_cypher("""CREATE INDEX point_row_col IF NOT EXISTS
FOR (p:Point) ON (p.row, p.col)""")

In [ ]:
input_file = "input.txt"

In [ ]:
lines = [{'ix':i, 'line':l[:-1]} for i, l in enumerate(open(input_file, 'r').readlines(), 1)]

gds.run_cypher("""
UNWIND $lines AS line
WITH line.ix AS ix, line.line AS line
CALL {
WITH ix, line
WITH ix, split(line,'') AS line
WITH [col IN range(0,size(line)-1) | {row:ix, col:col+1, c:line[col]}] AS line
UNWIND line AS point
CREATE (p:Point:Cell) SET p = point
} IN TRANSACTIONS OF 10000 ROWS
""", {'lines':lines})

In [ ]:
gds.run_cypher("""
MATCH (p:Point&Cell)
CALL {
WITH p
MERGE (n:Point:Port {col:p.col, row:p.row - 0.5})
MERGE (s:Point:Port {col:p.col, row: p.row + 0.5})
MERGE (e:Point:Port {col:p.col + 0.5, row: p.row})
MERGE (w:Point:Port {col:p.col - 0.5, row: p.row})
MERGE (p)<-[:IS_PORT_OF {dir:'N'}]-(n)
MERGE (p)<-[:IS_PORT_OF {dir:'S'}]-(s)
MERGE (p)<-[:IS_PORT_OF {dir:'E'}]-(e)
MERGE (p)<-[:IS_PORT_OF {dir:'W'}]-(w)
} IN TRANSACTIONS OF 1000 ROWS
""")

In [ ]:
query = """MATCH (in_port)-[:IS_PORT_OF {dir:$in_dir}]->(c:Point:Cell {c:$sym})
                <-[:IS_PORT_OF {dir:$out_dir}]-(out_port)
                MERGE (in_port)-[:CONNECTS {from_cell:elementId(c)}]->(out_port)"""
for sym in edges:
    for in_dir in edges[sym]:
        for out_dir in edges[sym][in_dir]:
            gds.run_cypher(query, {'sym':sym, 'in_dir': in_dir, 'out_dir': out_dir})


In [ ]:
gds.run_cypher("""
MATCH (p:Point {col:1, row:1})<-[:IS_PORT_OF {dir:'W'}]->(source:Port)
SET source:Source
""")

In [ ]:
gds.run_cypher("""
MATCH (p:Port)
WHERE EXISTS {(s:Source)-[rs:CONNECTS]->*(p)}
WITH DISTINCT p
MATCH (p)-[:IS_PORT_OF]->(c:Cell)
WITH DISTINCT c
RETURN count(c) AS part1
""")

In [ ]:
# alternative part 1
gds.run_cypher("""
MATCH (s:Source)
SET s:Lit
""")
gds.run_cypher("""
CALL apoc.periodic.commit('
MATCH (:Port&Lit)-[:CONNECTS]->(p:Port&!Lit)
SET p:Lit
WITH count(p) AS limit
RETURN limit
', {})
""")
gds.run_cypher("""
MATCH (c:Cell)
WHERE EXISTS {(c)<-[:IS_PORT_OF]-(:Lit)}
RETURN count(c) AS part1
""")


# Part 2

In [ ]:
gds.run_cypher("""
MATCH (p:Port)
WITH p, count {(p)-[:IS_PORT_OF]->()} AS deg
WHERE deg = 1
SET p:Edge""")

In [ ]:
gds.run_cypher("""
CALL apoc.periodic.iterate(
'MATCH (s:Edge)
RETURN s',
'MATCH (cell:Cell)
WHERE EXISTS {(s)-[:CONNECTS|IS_PORT_OF]->*(cell)}
WITH s, count(cell) AS energy
SET s.energy = energy',
{batchSize:1, parallel:TRUE})
""")

In [ ]:
gds.run_cypher("""
MATCH (e:Edge) WHERE NOT e.energy IS NULL
RETURN max(e.energy) AS part2
""")